In [1]:

from sklearn.model_selection import train_test_split
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import pandas as pd
import numpy as np
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Activation, Dropout, Flatten, Dense
import tensorflow as tf
from os import listdir
from PIL import Image
import glob, os

from os.path import isfile, join
import numpy as np
from pandas import DataFrame
import pandas as pd
from matplotlib import pyplot as plt
#from __future__ import print_function

from SRC.transform import *
#estas son mis funciones traidas del jupyter contiguo

In [2]:
paths={"black":"INPUT/black/",
      "white":"INPUT/white/",
      "green":"INPUT/green/",
      "red":"INPUT/red/"}
images={}
for color,path in paths.items():
    images[color]=[]
    for im in listdir(path):
        if im.endswith("jpg"):
            #size=(64,64)
            a=resize_image(path+im,(64,64))
            images[color].append(np.asarray(a))
    images[color]=np.array(images[color])

In [3]:
X= np.concatenate(list(images.values()),axis=0)
y = np.concatenate([[k]* v.shape[0] for k,v in images.items()])

# Division entre la x y la y para test
### Ojo, normalmente convertiriamos el onehotencoder antes de esta division para ahorrarnos un paso

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [5]:
img_width, img_height = 64, 64
img_rows, img_cols = 64, 64

# Prepare data to feed the NN
num_classes = 4

# Ask keras which format to use depending on used backend and arrange data as expected
if K.image_data_format() == 'channels_first':
    X_train = x_train.reshape(X_train.shape[0], 3, img_rows, img_cols)
    X_test = x_test.reshape(X_test.shape[0], 3, img_rows, img_cols)
    input_shape = (3, img_width, img_height)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 3)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 3)
    input_shape = (img_width, img_height, 3)

# Incoming data is in uint8. Cast the input data images to be floats in range [0.0-1.0]  
X_train = tf.keras.utils.normalize(X_train,axis=1)
X_test = tf.keras.utils.normalize(X_test,axis=1)
#X_train = X_train.astype('float32') / 255
#X_test = X_test.astype('float32') / 255

print('x_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

x_train shape: (8388, 64, 64, 3)
8388 train samples
2097 test samples


In [6]:
#red neuronal propiamente dicha

In [7]:
img_width, img_height = 64, 64

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(tf.keras.layers.Flatten())

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(4))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

### Proceso de cambio de las y para poder utilizarlas en el modelo. 
### Las necesitamos en el formato de array sin strings y getdummies no nos sirvio ya que 
### no podiamos obtener el color con posterioridad.

In [8]:
# importo aqui para que quede reflejado el modulo que utilizamos
from sklearn.preprocessing import OneHotEncoder

In [9]:
y_test


array(['black', 'white', 'black', ..., 'white', 'white', 'black'],
      dtype='<U5')

In [10]:
# Hacemos el reshape, luego onehot encoder, luego fit y por ultimo toarray
y_test= y_test.reshape(-1, 1)
y_test

array([['black'],
       ['white'],
       ['black'],
       ...,
       ['white'],
       ['white'],
       ['black']], dtype='<U5')

In [11]:
##3
enc = OneHotEncoder()
enc.fit(y_test)  

OneHotEncoder()

In [12]:
y_test = enc.transform(y_test).toarray()

In [13]:
#por fin y_test esta en la forma que queremos
y_test


array([[1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       ...,
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.]])

In [14]:
#ahora hago lo mismo con y Y_train

In [15]:
y_train

array(['black', 'green', 'white', ..., 'white', 'red', 'white'],
      dtype='<U5')

In [16]:
y_train= y_train.reshape(-1, 1)

In [17]:
enc = OneHotEncoder()
enc.fit(y_train)  

OneHotEncoder()

In [18]:
y_train = enc.transform(y_train).toarray()

In [19]:
y_train

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       ...,
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [20]:
#ha funcionado. Tenemos y_test e y_train en el formato que necesitabamos
# podiamos haberlo hecho antes de dividir el split del train pero he decidido hacerlo dos veces para practicar

In [21]:
enc.get_feature_names()

array(['x0_black', 'x0_green', 'x0_red', 'x0_white'], dtype=object)

In [22]:
enc.inverse_transform([[0,1,0,0]])

array([['green']], dtype='<U5')

In [23]:
# Codigo para para correr el modelo de red neuronal. 

In [24]:
batch_size = 100
epochs = 20

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test))



Epoch 1/20
84/84 [==============================] - 12s 145ms/step - loss: 0.3419 - accuracy: 0.7114 - val_loss: 0.1902 - val_accuracy: 0.8522
Epoch 2/20
84/84 [==============================] - 13s 151ms/step - loss: 0.2038 - accuracy: 0.8501 - val_loss: 0.1488 - val_accuracy: 0.8784
Epoch 3/20
84/84 [==============================] - 14s 164ms/step - loss: 0.1700 - accuracy: 0.8763 - val_loss: 0.1196 - val_accuracy: 0.9075
Epoch 4/20
84/84 [==============================] - 14s 168ms/step - loss: 0.1399 - accuracy: 0.9019 - val_loss: 0.0934 - val_accuracy: 0.9299
Epoch 5/20
84/84 [==============================] - 15s 173ms/step - loss: 0.1081 - accuracy: 0.9231 - val_loss: 0.0708 - val_accuracy: 0.9537
Epoch 6/20
84/84 [==============================] - 14s 171ms/step - loss: 0.0880 - accuracy: 0.9437 - val_loss: 0.0615 - val_accuracy: 0.9599
Epoch 7/20
84/84 [==============================] - 14s 169ms/step - loss: 0.0709 - accuracy: 0.9529 - val_loss: 0.0503 - val_accuracy: 0.9695

In [ ]:
# PRUEBAS

In [25]:
X_test[45].shape
model.predict(np.array([X_test[2000]]))
#nos dice cual es la prediccion del modelo

array([[2.3661310e-14, 1.0887863e-12, 1.0000000e+00, 3.4082523e-16]],
      dtype=float32)

In [26]:
enc.get_feature_names()
# y hay que recordar que los colores marcados son:

array(['x0_black', 'x0_green', 'x0_red', 'x0_white'], dtype=object)

In [27]:
enc.inverse_transform([[9.9999976e-01, 6.4836332e-09, 1.1157968e-09, 1.2618054e-12]])
#nos da el resultado mas probable en una tuple. 


array([['black']], dtype='<U5')

In [28]:
X[2000].shape

(64, 64, 3)

In [29]:
model.predict(np.array([X_test[2000]]))

array([[2.3661310e-14, 1.0887863e-12, 1.0000000e+00, 3.4082523e-16]],
      dtype=float32)

In [30]:
enc.inverse_transform([[9.9999976e-01, 6.4836332e-09, 1.1157968e-09, 1.2618054e-12]])[0][0]
# le digo que me muestre solo el primer elemento, que es el color

'black'

In [38]:
X_test[2000].size

12288

In [31]:
X_test[2000]

array([[[0.16860769, 0.17039556, 0.16977151],
        [0.16208582, 0.16408352, 0.16413284],
        [0.18283584, 0.19340441, 0.19027679],
        ...,
        [0.10889563, 0.12785336, 0.12889548],
        [0.14306867, 0.1496268 , 0.15035371],
        [0.12678506, 0.13105385, 0.13056816]],

       [[0.16271232, 0.16809292, 0.16639857],
        [0.16091973, 0.16636246, 0.16524939],
        [0.18955774, 0.1947112 , 0.19413375],
        ...,
        [0.11635423, 0.13637692, 0.13754619],
        [0.13325824, 0.13959595, 0.14021751],
        [0.13838631, 0.14274018, 0.14228582]],

       [[0.16978677, 0.17039556, 0.16977151],
        [0.16675016, 0.16750193, 0.16748249],
        [0.19493527, 0.1947112 , 0.19541941],
        ...,
        [0.11486251, 0.13467221, 0.13581605],
        [0.1299881 , 0.13625234, 0.13683877],
        [0.13755765, 0.14190545, 0.14144885]],

       ...,

       [[0.05777466, 0.06102003, 0.06296162],
        [0.05597208, 0.05925238, 0.06141025],
        [0.06318591, 0

In [32]:
def car_colour (im):
    x = model.predict(np.array([im]))
    
    return x[0]

In [33]:
print (car_colour (X[600]))

[1. 0. 0. 0.]


In [34]:
enc.inverse_transform([[1. 0. 0. 0.]])[0][0]

SyntaxError: invalid syntax (<ipython-input-34-3e238f67da9e>, line 1)

In [35]:
# las imagenes deben ser metidas como el array de arriba, de X[2000]
def car_colour (im):
    x = model.predict(np.array([im]))
    y = enc.inverse_transform([x[0]])[0][0]
    if y=='black':
        frase
        return f'The color of the car of the picture is {y}, like my soul'
    elif y== "red":
        return f'The color of the car of the picture is {y}, like the communist you are'
    elif y== "yellow":
        return f'The color of the car of the picture is {y}, like "hielo" in english'
    elif y=="white":
        return f"The color of the car of the picture is {y}, like the white russian I'll have this evening"

In [36]:
print (car_colour (X[7002]))

The color of the car of the picture is white, like the white russian I'll have this evening


In [37]:
X.shape

(10485, 64, 64, 3)

In [ ]:
# Que alguien me explique porque funciona cuando lo hago todo del tiron en la misma funcion 
# pero no cuando lo hago por partes.

#le puedo poner un random choice para que diga distintas estupideces al azar segun el color

# 